## Fake News using Artificial Immune Systems 

### Data

✅ politifact_real.csv → real articles from Politifact 

✅ politifact_fake.csv → fake articles from Politifact

✅ gossipcop_real.csv → real articles from GossipCop

✅ gossipcop_fake.csv → fake articles from GossipCop

### 📚 [References](https://github.com/KaiDMML/FakeNewsNet)

- Shu, K., Mahudeswaran, D., Wang, S., Lee, D., & Liu, H. (2018). **FakeNewsNet: A Data Repository with News Content, Social Context and Dynamic Information for Studying Fake News on Social Media.** *arXiv preprint arXiv:1809.01286*. [arXiv link](https://arxiv.org/abs/1809.01286)

- Shu, K., Sliva, A., Wang, S., Tang, J., & Liu, H. (2017). **Fake News Detection on Social Media: A Data Mining Perspective.** *ACM SIGKDD Explorations Newsletter*, 19(1), 22–36. [DOI](https://doi.org/10.1145/3137597.3137600)

- Shu, K., Wang, S., & Liu, H. (2017). **Exploiting Tri-Relationship for Fake News Detection.** *arXiv preprint arXiv:1712.07709*. [arXiv link](https://arxiv.org/abs/1712.07709)
✅ Includes 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Load real and fake from politifact
basepath = "/Users/ayeshamendoza/repos/fake-news-immune-system"
datapath = os.path.join(basepath, "data/raw")
real = pd.read_csv(os.path.join(datapath, 'politifact_real.csv'))
fake = pd.read_csv(os.path.join(datapath, 'politifact_fake.csv'))

print("Real news shape:", real.shape)
print("Fake news shape:", fake.shape)

print("\nSample real news article:")
print(real.iloc[0])

print("\nSample fake news article:")
print(fake.iloc[0])

Real news shape: (624, 4)
Fake news shape: (432, 4)

Sample real news article:
id                                             politifact14984
news_url                             http://www.nfib-sbet.org/
title              National Federation of Independent Business
tweet_ids    967132259869487105\t967164368768196609\t967215...
Name: 0, dtype: object

Sample fake news article:
id                                             politifact15014
news_url             speedtalk.com/forum/viewtopic.php?t=51650
title        BREAKING: First NFL Team Declares Bankruptcy O...
tweet_ids    937349434668498944\t937379378006282240\t937380...
Name: 0, dtype: object


Data Preprocessing

In order to be able to use the text data in our Deep Learning models, we will need to convert the text data to numbers.  In order to do that the following pre-processing steps were done:

- Tokenization
- Stemming
- removing stopwords
- removing punctuations
- TF-IDF

In [3]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# Download NLTK resources if not done
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

# Define cleaning function
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer
import string
import re



In [4]:
def clean_text(text):
    stop_words = ENGLISH_STOP_WORDS
    stemmer = PorterStemmer()

    text = text.lower()
    text = re.sub(r'\.{2,}', ' ', text)              # remove ellipsis
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text) # remove URLs
    text = re.sub(r'\$\w*', '', text)                # remove $ mentions
    text = re.sub(r'#', '', text)                    # remove hashtags
    text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)  # <-- remove punctuation

    tokens = text.split()  # now safe to split on whitespace

    cleaned_tokens = [
        stemmer.stem(token)
        for token in tokens
        if token not in stop_words
    ]

    return ' '.join(cleaned_tokens)



# Load dataset
basepath = "/Users/ayeshamendoza/repos/fake-news-immune-system"
datapath = os.path.join(basepath, "data/raw")
real = pd.read_csv(os.path.join(datapath, 'politifact_real.csv'))
fake = pd.read_csv(os.path.join(datapath, 'politifact_fake.csv'))

# Add label columns
real['label'] = 'REAL'
fake['label'] = 'FAKE'

# Combine datasets
df = pd.concat([real, fake], ignore_index=True)



In [6]:
!pip install spacy

  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached rich-14.0.0-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)

In [7]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 12.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [8]:
# Apply cleaning
df['clean_text'] = df['title'].fillna('')
df['clean_text'] = df['clean_text'].apply(clean_text)
# df['clean_text'] = df['title'].fillna('').apply(clean_text)

# OPTIONAL: Save cleaned dataset
df.to_csv('../data/processed/cleaned_articles.csv', index=False)

# Preview cleaned text
print(df[['label', 'clean_text']].head())

article_texts = df['clean_text'].tolist()

# # ✅ TF-IDF Vectorizer
# vectorizer = TfidfVectorizer(max_features=5000)
# tfidf_matrix = vectorizer.fit_transform(df['clean_text'])

# print("TF-IDF matrix shape:", tfidf_matrix.shape)




  label                                         clean_text
0  REAL                         nation feder independ busi
1  REAL                              comment fayettevil nc
2  REAL  romney make pitch hope close deal elect rocki ...
3  REAL  democrat leader say hous democrat unit gop def...
4  REAL                   budget unit state govern fy 2008


In [9]:
import spacy
import numpy as np

nlp = spacy.load("en_core_web_md")

article_vectors = []
for doc in nlp.pipe(article_texts, disable=["ner", "parser"]):
    article_vectors.append(doc.vector)

article_vectors = np.array(article_vectors)
print("✅ Embeddings shape:", article_vectors.shape)

✅ Embeddings shape: (1056, 300)


In [14]:
import sys
sys.path.append('../') 
import src.negative_selection
import importlib
importlib.reload(src.negative_selection)

from src.negative_selection import generate_detectors, detect_anomaly
# From your DataFrame
# article_vectors, true_labels = embed_articles_from_df(df, text_col='text', label_col='label')
label_map = {'REAL': 0, 'FAKE': 1}
true_labels = df['label'].map(label_map).tolist()
# Use only real news vectors for training
num_real = sum(1 for label in true_labels if label == 0)
self_matrix = article_vectors[:num_real]
vector_dim = article_vectors.shape[1]

# Generate detectors
num_detectors = 300 #100
detectors = generate_detectors(
    num_detectors=num_detectors,
    vector_dim=vector_dim,
    self_matrix=self_matrix,
    threshold=0.4,
    noise_std=0.05
)


✅ Generated 300 detectors in 1821 attempts (threshold=0.4, noise_std=0.05)


In [15]:
from sklearn.metrics.pairwise import cosine_distances
import numpy as np

all_min_dists = []
for article_vec in article_vectors:
    distances = cosine_distances(detectors, article_vec.reshape(1, -1)).flatten()
    all_min_dists.append(np.min(distances))

print("🔎 Min distance:", np.min(all_min_dists))
print("📏 Max distance:", np.max(all_min_dists))
print("📊 Mean distance:", np.mean(all_min_dists))


🔎 Min distance: 0.40016254506388793
📏 Max distance: 1.0
📊 Mean distance: 0.5693185149347891


✅ Your Detector-to-Article Distance Stats:
Metric	Value	What It Means
Min	0.4001	One article is just barely triggering a detector at threshold = 0.4 — perfect edge case for detection!
Mean	0.59	Most articles are comfortably distant — ideal for anomaly detection
Max	1.0	Some articles are totally outside detector space, as expected

🔥 What This Tells Us:
Your detectors are nicely placed in embedding space

Using a threshold of 0.4–0.6 should give actual detection coverage now

You're ready to test the full prediction + evaluation loop

In [20]:
from IPython.display import display, HTML

# This disables output scrolling in notebook cells
display(HTML("<style>.output_wrapper, .output {height:auto !important; max-height: none !important;}</style>"))


In [22]:
savepath

NameError: name 'savepath' is not defined

In [23]:
from sklearn.metrics import classification_report, confusion_matrix
savepath = os.path.join(basepath, "data/processed")

writepath = os.path.join(savepath, "ais_results.txt")
thresholds = [0.5, 0.52, 0.54, 0.56, 0.58, 0.6]

with open(writepath, "w") as f:
    for t in thresholds:
        predictions = []
        for vec in article_vectors:
            distances = cosine_distances(detectors, vec.reshape(1, -1)).flatten()
            is_fake = np.any(distances < t)
            predictions.append(int(is_fake))

        f.write(f"\n🔍 Threshold = {t}\n")
        f.write(str(confusion_matrix(true_labels, predictions)) + "\n")
        f.write(classification_report(true_labels, predictions, target_names=["Real", "Fake"]))


19

23

326

20

23

...

23

326

19

23

326

In [21]:
from sklearn.metrics import classification_report, confusion_matrix

thresholds = [0.5, 0.52, 0.54, 0.56, 0.58, 0.6]
# for t in [0.4, 0.45, 0.5, 0.55]:
for t in thresholds:
    predictions = []
    for vec in article_vectors:
        distances = cosine_distances(detectors, vec.reshape(1, -1)).flatten()
        is_fake = np.any(distances < t)
        predictions.append(int(is_fake))

    print(f"\n🔍 Threshold = {t}")
    print(confusion_matrix(true_labels, predictions))
    print(classification_report(true_labels, predictions, target_names=["Real", "Fake"]))



🔍 Threshold = 0.5
[[448 176]
 [403  29]]
              precision    recall  f1-score   support

        Real       0.53      0.72      0.61       624
        Fake       0.14      0.07      0.09       432

    accuracy                           0.45      1056
   macro avg       0.33      0.39      0.35      1056
weighted avg       0.37      0.45      0.40      1056


🔍 Threshold = 0.52
[[407 217]
 [374  58]]
              precision    recall  f1-score   support

        Real       0.52      0.65      0.58       624
        Fake       0.21      0.13      0.16       432

    accuracy                           0.44      1056
   macro avg       0.37      0.39      0.37      1056
weighted avg       0.39      0.44      0.41      1056


🔍 Threshold = 0.54
[[349 275]
 [335  97]]
              precision    recall  f1-score   support

        Real       0.51      0.56      0.53       624
        Fake       0.26      0.22      0.24       432

    accuracy                           0.42      1056


In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer


# # Fit TF-IDF
# # vectorizer = TfidfVectorizer(max_features=5000)  # limit vocab to top 5000 tokens
# vectorizer = TfidfVectorizer(
#     max_features=5000,
#     token_pattern=r'(?u)\b[a-zA-Z]{2,}\b'
# )
# tfidf_matrix = vectorizer.fit_transform(df['clean_text'])

# # Vocab size
# print(f"Vocabulary size: {len(vectorizer.vocabulary_)}")

# # Preview first 20 tokens in vocab
# print("\nSample vocab tokens:")
# sample_tokens = list(vectorizer.vocabulary_.keys())[:20]
# print(sample_tokens)

# # Show shape
# print(f"\nTF-IDF matrix shape: {tfidf_matrix.shape}")

# # Show top tokens by IDF (most unique)
# idf_scores = vectorizer.idf_
# tokens_idf = sorted(zip(vectorizer.get_feature_names_out(), idf_scores), key=lambda x: -x[1])
# print("\nTop 10 tokens by IDF (most unique):")
# for token, idf in tokens_idf[:10]:
#     print(f"{token}: {idf:.2f}")


In [ ]:
basepath

In [ ]:
from scipy import sparse

savepath = os.path.join(basepath, "data/processed")
df.to_csv(os.path.join(savepath, "clean_articles.csv"), index=False)

sparse.save_npz(os.path.join(savepath, "tfidf_matrix.npz"), tfidf_matrix)

# Save vocab
import pickle
with open(os.path.join(savepath,'tfidf_vocab.pkl'), 'wb') as f:
    pickle.dump(vectorizer.vocabulary_, f)

In [ ]:
import sys
sys.path.append('../') 
import src.negative_selection
import importlib
importlib.reload(src.negative_selection)

from src.negative_selection import generate_detectors
from scipy import sparse

# Load saved tfidf matrix
self_matrix = sparse.load_npz(os.path.join(savepath, 'tfidf_matrix.npz'))

threshold = 0.8  # tweak threshold as needed
num_detectors = 100

num_real = len(real)
self_matrix = tfidf_matrix[:num_real]  # ONLY real news
vector_dim = self_matrix.shape[1]

# detectors = generate_detectors(num_detectors, vector_dim, self_matrix, threshold)
# detectors = generate_detectors(200, vector_dim, self_matrix, threshold)
detectors = generate_detectors(
    num_detectors=100,
    vector_dim=vector_dim,
    self_matrix=self_matrix,
    threshold=0.5,         # More realistic
    noise_std=0.05         # Gives variation
)


print(f"Generated {len(detectors)} detectors (requested {num_detectors})")

In [ ]:
print(detectors.shape)  # should be (100, vector_dim)
print(detectors[0][:10])  # first 10 values of first detector


In [ ]:
# Debug
import numpy as np
from sklearn.metrics.pairwise import cosine_distances

all_min_dists = []

for article_vec in tfidf_matrix.toarray():  # or .A
    distances = cosine_distances(detectors, article_vec.reshape(1, -1)).flatten()
    all_min_dists.append(np.min(distances))

# Basic stats
print("Min distance:", np.min(all_min_dists))
print("Max distance:", np.max(all_min_dists))
print("Mean distance:", np.mean(all_min_dists))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

predictions = []
for article_vec in tfidf_matrix.toarray():
    distances = cosine_distances(detectors, article_vec.reshape(1, -1)).flatten()
    is_fake = np.any(distances < 0.55)  # ← threshold here
    predictions.append(int(is_fake))

# Evaluate
label_map = {'REAL': 0, 'FAKE': 1}
true_labels = df['label'].map(label_map).tolist()
print(confusion_matrix(true_labels, predictions))
print(classification_report(true_labels, predictions, target_names=["Real", "Fake"]))


🔍 Let’s Break Down What’s Going On
Confusion Matrix:


[[509 115]   ← Real: 509 correct, 115 false positives (flagged as fake)
 [432   0]]   ← Fake: 432 fake articles, all missed ❌


We're correctly identifying a good number of real articles (recall = 82%)

But we're not catching any fake news at all — detectors didn’t fire on them

Precision for "Fake" = 0, recall for "Fake" = 0 → F1 = 0

💡 Diagnosis
❓Possibility 1: Detectors are too similar to real, not close to fake
We built detectors based on noise from real news

If fake news vectors look too similar to real ones (in TF-IDF space), they slip through undetected

❓Possibility 2: Threshold is too strict
You used threshold = 0.55

But we saw earlier that min distances start at ~0.50, and mean = 0.84

Try lowering threshold to ~0.7 or even 0.75 to allow detectors to fire on fake news

In [ ]:
for t in [0.7, 0.75]:
    predictions = []
    for article_vec in tfidf_matrix.toarray():
        distances = cosine_distances(detectors, article_vec.reshape(1, -1)).flatten()
        is_fake = np.any(distances < t)
        predictions.append(int(is_fake))

    print(f"\n🔎 Threshold = {t}")
    print(confusion_matrix(true_labels, predictions))
    print(classification_report(true_labels, predictions, target_names=["Real", "Fake"]))


In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# sample_detector = detectors[0]
# distances = np.linalg.norm(self_matrix.toarray() - sample_detector, axis=1)

# plt.hist(distances, bins=30)
# plt.xlabel('Distance to self')
# plt.ylabel('Count')
# plt.title('Distances from sample detector to self samples')
# plt.show()


In [ ]:
import sys
sys.path.append('../') 

import src.negative_selection
import importlib
importlib.reload(src.negative_selection)

from src.negative_selection import detect_anomaly

# # Pick sample article (convert sparse to dense row)
# sample_article_vector = tfidf_matrix[0].toarray()[0]

# result = detect_anomaly(sample_article_vector, detectors, threshold)

# print("Article detected as FAKE" if result else "Article detected as REAL")

predictions = []
threshold = 0.05

for i in range(tfidf_matrix.shape[0]):
    # Get article vector → convert sparse row to dense array
    article_vector = tfidf_matrix[i].toarray()[0]
    
    # Run detection
    detected = detect_anomaly(article_vector, detectors, threshold)
    
    # Map True/False → FAKE/REAL
    predictions.append('FAKE' if detected else 'REAL')

# Assign predictions to dataframe
df['predicted_label'] = predictions


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(df['label'], predictions, target_names=["Real", "Fake"]))
print(confusion_matrix(df['label'], predictions))


In [ ]:
accuracy = (df['label'] == df['predicted_label']).mean()
print(f"Accuracy: {accuracy:.2%}")

In [ ]:
for t in [0.02, 0.04, 0.08]:
    preds = []
    for i in range(tfidf_matrix.shape[0]):
        article_vector = tfidf_matrix[i].toarray()[0]
        detected = detect_anomaly(article_vector, detectors, t)
        preds.append('FAKE' if detected else 'REAL')
    acc = (df['label'] == preds).mean()
    print(f"Threshold {t}: Accuracy {acc:.2%}")
